In [93]:
# TODO
# 1. # of words
# 2. # of sensor types
# 3. how bag of words clustering works
# 4. how data feature classification works on sensor types
# 5. how data feature classification works on tag classification
# 6. # of unique sentence structure

In [107]:
import json
from functools import reduce
import os.path
import os
import random

import pandas as pd
from scipy.stats import entropy
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessClassifier

In [46]:
nae_dict = {
    'bonner': ['607', '608', '609', '557', '610'],
    'ap_m': ['514', '513','604'],
    'bsb': ['519', '568', '567', '566', '564', '565'],
    'ebu3b': ['505', '506']
}

In [56]:
def counterize_feature(feat):
        indexList = [not np.isnan(val) for val in feat]
        maxVal = max(feat.loc[indexList])
        minVal = min(feat.loc[indexList])
        gran = 100
        interval = (maxVal-minVal)/100.0
        keys = np.arange(minVal,maxVal,interval)
        resultDict = defaultdict(int)
        for key, val in feat.iteritems():
            try:
                if np.isnan(val):
                    resultDict[None] += 1
                    continue
                diffList = [abs(key-val) for key in keys]
                minVal = min(diffList)
                minIdx = diffList.index(minVal)
                minKey = keys[minIdx]
                resultDict[minKey] += 1
            except:
                print key, val
        return resultDict

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-56-fc45969a359f>, line 20)

In [73]:
true_df.loc[true_df['Unique Identifier']=='505_0_3000003']['Schema Label'].ravel()[0]
#sklearn.ensemble.RandomForestClassifier

'Hot Water Return Temperature'

In [137]:
building_list = ['ebu3b']
for building_name in building_list:
    print("============ %s ==========="%building_name)
    with open('metadata/%s_sentence_dict.json'%building_name, 'r') as fp:
        sentence_dict = json.load(fp)
    srcid_list = list(sentence_dict.keys())
        
    # 1. Number of unique words
    adder = lambda x,y:x+y
    num_remover = lambda xlist: ["number" if x.isdigit() else x for x in xlist]
    total_word_set = set(reduce(adder, map(num_remover,sentence_dict.values()), []))

    print("# of unique words: %d"%(len(total_word_set)))

    # 2. of sensor types
    labeled_metadata_filename = 'metadata/%s_sensor_types_location.csv'%building_name
    if os.path.isfile(labeled_metadata_filename):
        true_df = pd.read_csv(labeled_metadata_filename)
    else:
        true_df = None

    if isinstance(true_df, pd.DataFrame):
        sensor_type_set = set(true_df['Schema Label'].ravel())
        print("# of unique sensor types: %d"%(len(sensor_type_set)))
    else:
        sensor_type_set = None

    # 3. how bag of words clustering works
    
    with open('model/%s_word_clustering.json'%building_name, 'r') as fp:
        cluster_dict = json.load(fp)
    print("# of word clusterings: %d"%(len(cluster_dict)))
    small_cluster_num = 0
    large_cluster_num = 0
    for cluster_id, srcids in cluster_dict.items():
        if len(srcids)<5:
            small_cluster_num +=1
        else:
            large_cluster_num +=1
    print("# of word small (<5)clusterings: %d"%small_cluster_num)
    print("# of word large (>=5)clusterings: %d"%large_cluster_num)
    
    # 4. how data feature classification works on sensor types
    with open('model/fe_%s.json'%building_name, 'r') as fp:
        #data_feature_dict = json.load(fp)
        pass
    with open('model/fe_%s_normalized.json'%building_name, 'r') as fp:
        data_feature_dict = json.load(fp)
        pass
    feature_num = len(list(data_feature_dict.values())[0])
    data_available_srcid_list = list(data_feature_dict.keys())
    if isinstance(true_df, pd.DataFrame):
        sample_num = 500
        sample_idx_list = random.sample(range(0,len(data_feature_dict)), sample_num)
        learning_srcid_list = [data_available_srcid_list[sample_idx]
                               for sample_idx in sample_idx_list]
        learning_x = [data_feature_dict[srcid] for srcid in learning_srcid_list]
        learning_y = [true_df.loc[true_df['Unique Identifier']==srcid]
                      ['Schema Label'].ravel()[0]
                      for srcid in learning_srcid_list]
        test_srcid_list = [srcid for srcid in data_available_srcid_list 
                           if srcid not in learning_srcid_list]
        test_x = [data_feature_dict[srcid] for srcid in test_srcid_list]
        
        classifier_list = [RandomForestClassifier(),
                           AdaBoostClassifier(),
                           MLPClassifier(),
                           KNeighborsClassifier(),
                           SVC(),
                           GaussianNB(),
                           DecisionTreeClassifier()
                          ]
        for classifier in classifier_list:
            classifier.fit(learning_x, learning_y)
            test_y = classifier.predict(test_x)
            precision = calc_accuracy(test_srcid_list, test_y)
            print(type(classifier).__name__, precision)
    
    # 5. How entropy varies in clusters
    entropy_dict = dict()
    for cluster_id, cluster in cluster_dict.items():
        entropy_list = list()
        for feature_idx in range(0,feature_num):
            entropy_list.append(\
                entropy([data_feature_dict[srcid][feature_idx] + 0.01
                         for srcid in cluster #random_sample_srcid_list \
                         if srcid in data_available_srcid_list]))
        entropy_dict[cluster_id] = entropy_list

    # 5. how data feature classification works on tag classification
    
    
    #if isinstance()
    # 6. # of unique sentence structure

============ ebu3b ===========
# of unique words: 438
# of unique sensor types: 164
# of word clusterings: 348
# of word small (<5)clusterings: 311
# of word large (>=5)clusterings: 37
RandomForestClassifier 0.5487595185458118
AdaBoostClassifier 0.12576762466224514
MLPClassifier 0.12306558585114223
KNeighborsClassifier 0.33407025300908866
SVC 0.18447555883075412
GaussianNB 0.414394497666421


KeyboardInterrupt: 

In [132]:
def feature_check(data_feature_dict):
    for srcid, features in data_feature_dict.items():
        for feat in features:
            #if np.isnan(feat):
            if feat < -100:
                print(srcid, features)

In [88]:
correct_cnt = 0
for i, srcid in enumerate(test_srcid_list):
    schema_label = true_df.loc[true_df['Unique Identifier']==srcid]['Schema Label'].ravel()[0]
    if schema_label==test_y[i]:
        correct_cnt += 1
print(correct_cnt)
print(correct_cnt/len(test_srcid_list))

1975
0.5530663679641556
